In [1]:
# Dependencies and data.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import tensorflow as tf

df = pd.read_csv('Resources/loan_status.csv')
df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
# Check unique values for bucketing needs.
cats = df.dtypes[df.dtypes == 'object'].index.tolist()
df[cats].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts of the Years_in_current_job variable.
df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [4]:
# Encode categorical variables.
enc = OneHotEncoder(sparse=False)

# Fit and transform the encoder to a DataFrame, fix column names.
encode_df = pd.DataFrame(enc.fit_transform(df[cats]))
encode_df.columns = enc.get_feature_names(cats)

encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# Merge together encoded and orignal DF, dropping original cartegorical variables.
df = df.merge(encode_df, left_index=True, right_index=True).drop(cats, 1)
df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Split and standardize the data.
y = df.Loan_Status_Fully_Paid
X = df.drop(columns=['Loan_Status_Fully_Paid', 'Loan_Status_Not_Paid'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Initialize scaler object.
X_scaler = StandardScaler().fit(X_train)

# Transform data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# RandomForest model.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Train the model.
rf_model.fit(X_train_scaled, y_train)

# Evaluate the model.
y_pred = rf_model.predict(X_test_scaled)
print(f'Random Forest Classifier Accuracy Score: {accuracy_score(y_test, y_pred)}')

Random Forest Classifier Accuracy Score: 0.8491104766088293


In [8]:
# Neural Network.
nn = tf.keras.models.Sequential()

# Add layers.
nn.add(tf.keras.layers.Dense(input_dim=len(X_train_scaled[0]), units=24, activation='relu'))
nn.add(tf.keras.layers.Dense(units=12, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
# Check model shape.
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                888       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Import checkpoint dependencies.
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames.
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [11]:
# Compile model.
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create the callback Checkpoint function.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100
)

# Train the model.
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

# Evaluate the model.
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Neural Network Loss: {model_loss}, and Accuracy: {model_accuracy}')

Epoch 1/100
 93/854 [==>...........................] - ETA: 1s - loss: 0.5237 - accuracy: 0.7950
Epoch 00001: saving model to checkpoints\weights.01.hdf5
191/854 [=====>........................] - ETA: 1s - loss: 0.4934 - accuracy: 0.8073
Epoch 00001: saving model to checkpoints\weights.01.hdf5
284/854 [========>.....................] - ETA: 1s - loss: 0.4659 - accuracy: 0.8241
Epoch 00001: saving model to checkpoints\weights.01.hdf5
399/854 [=============>................] - ETA: 1s - loss: 0.4472 - accuracy: 0.8311
Epoch 00001: saving model to checkpoints\weights.01.hdf5
489/854 [================>.............] - ETA: 0s - loss: 0.4373 - accuracy: 0.8360
Epoch 00001: saving model to checkpoints\weights.01.hdf5
581/854 [===================>..........] - ETA: 0s - loss: 0.4347 - accuracy: 0.8350
Epoch 00001: saving model to checkpoints\weights.01.hdf5
692/854 [=======================>......] - ETA: 0s - loss: 0.4289 - accuracy: 0.8366
Epoch 00001: saving model to checkpoints\weights.01

252/854 [=======>......................] - ETA: 1s - loss: 0.3694 - accuracy: 0.8534
Epoch 00007: saving model to checkpoints\weights.07.hdf5
370/854 [===========>..................] - ETA: 1s - loss: 0.3757 - accuracy: 0.8493
Epoch 00007: saving model to checkpoints\weights.07.hdf5
465/854 [===============>..............] - ETA: 0s - loss: 0.3784 - accuracy: 0.8489
Epoch 00007: saving model to checkpoints\weights.07.hdf5
558/854 [==================>...........] - ETA: 0s - loss: 0.3790 - accuracy: 0.8485
Epoch 00007: saving model to checkpoints\weights.07.hdf5
674/854 [======================>.......] - ETA: 0s - loss: 0.3799 - accuracy: 0.8480
Epoch 00007: saving model to checkpoints\weights.07.hdf5
767/854 [=========================>....] - ETA: 0s - loss: 0.3780 - accuracy: 0.8494
Epoch 00007: saving model to checkpoints\weights.07.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3762 - accuracy: 0.8502
Epoch 8/100
  1/854 [..............................] - ETA:

440/854 [==============>...............] - ETA: 0s - loss: 0.3669 - accuracy: 0.8536
Epoch 00013: saving model to checkpoints\weights.13.hdf5
531/854 [=================>............] - ETA: 0s - loss: 0.3692 - accuracy: 0.8526
Epoch 00013: saving model to checkpoints\weights.13.hdf5
651/854 [=====================>........] - ETA: 0s - loss: 0.3696 - accuracy: 0.8520
Epoch 00013: saving model to checkpoints\weights.13.hdf5
743/854 [=========================>....] - ETA: 0s - loss: 0.3719 - accuracy: 0.8506
Epoch 00013: saving model to checkpoints\weights.13.hdf5
851/854 [============================>.] - ETA: 0s - loss: 0.3728 - accuracy: 0.8503
Epoch 00013: saving model to checkpoints\weights.13.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3728 - accuracy: 0.8502
Epoch 14/100
 76/854 [=>............................] - ETA: 1s - loss: 0.3691 - accuracy: 0.8479
Epoch 00014: saving model to checkpoints\weights.14.hdf5
196/854 [=====>........................] - ETA

613/854 [====================>.........] - ETA: 0s - loss: 0.3671 - accuracy: 0.8519
Epoch 00019: saving model to checkpoints\weights.19.hdf5
719/854 [========================>.....] - ETA: 0s - loss: 0.3683 - accuracy: 0.8520
Epoch 00019: saving model to checkpoints\weights.19.hdf5
810/854 [===========================>..] - ETA: 0s - loss: 0.3696 - accuracy: 0.8514
Epoch 00019: saving model to checkpoints\weights.19.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3701 - accuracy: 0.8512
Epoch 20/100
 73/854 [=>............................] - ETA: 1s - loss: 0.3778 - accuracy: 0.8442
Epoch 00020: saving model to checkpoints\weights.20.hdf5
167/854 [====>.........................] - ETA: 1s - loss: 0.3702 - accuracy: 0.8479
Epoch 00020: saving model to checkpoints\weights.20.hdf5
271/854 [========>.....................] - ETA: 1s - loss: 0.3674 - accuracy: 0.8507
Epoch 00020: saving model to checkpoints\weights.20.hdf5
361/854 [===========>..................] - ETA

797/854 [==========================>...] - ETA: 0s - loss: 0.3675 - accuracy: 0.8519
Epoch 00025: saving model to checkpoints\weights.25.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3678 - accuracy: 0.8514
Epoch 26/100
 26/854 [..............................] - ETA: 1s - loss: 0.3532 - accuracy: 0.8510
Epoch 00026: saving model to checkpoints\weights.26.hdf5
146/854 [====>.........................] - ETA: 1s - loss: 0.3544 - accuracy: 0.8577
Epoch 00026: saving model to checkpoints\weights.26.hdf5
239/854 [=======>......................] - ETA: 1s - loss: 0.3603 - accuracy: 0.8555
Epoch 00026: saving model to checkpoints\weights.26.hdf5
332/854 [==========>...................] - ETA: 1s - loss: 0.3647 - accuracy: 0.8517
Epoch 00026: saving model to checkpoints\weights.26.hdf5
427/854 [==============>...............] - ETA: 0s - loss: 0.3653 - accuracy: 0.8516
Epoch 00026: saving model to checkpoints\weights.26.hdf5
550/854 [==================>...........] - ETA

...

501/854 [================>.............] - ETA: 0s - loss: 0.3589 - accuracy: 0.8524
Epoch 00075: saving model to checkpoints\weights.75.hdf5
594/854 [===================>..........] - ETA: 0s - loss: 0.3574 - accuracy: 0.8540
Epoch 00075: saving model to checkpoints\weights.75.hdf5
683/854 [======================>.......] - ETA: 0s - loss: 0.3562 - accuracy: 0.8547
Epoch 00075: saving model to checkpoints\weights.75.hdf5
797/854 [==========================>...] - ETA: 0s - loss: 0.3574 - accuracy: 0.8532
Epoch 00075: saving model to checkpoints\weights.75.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3573 - accuracy: 0.8536
Epoch 76/100
 49/854 [>.............................] - ETA: 1s - loss: 0.3638 - accuracy: 0.8457
Epoch 00076: saving model to checkpoints\weights.76.hdf5
141/854 [===>..........................] - ETA: 1s - loss: 0.3569 - accuracy: 0.8511
Epoch 00076: saving model to checkpoints\weights.76.hdf5
232/854 [=======>......................] - ETA

675/854 [======================>.......] - ETA: 0s - loss: 0.3552 - accuracy: 0.8533
Epoch 00081: saving model to checkpoints\weights.81.hdf5
768/854 [=========================>....] - ETA: 0s - loss: 0.3568 - accuracy: 0.8526
Epoch 00081: saving model to checkpoints\weights.81.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3567 - accuracy: 0.8531
Epoch 82/100
 25/854 [..............................] - ETA: 1s - loss: 0.3567 - accuracy: 0.8600
Epoch 00082: saving model to checkpoints\weights.82.hdf5
120/854 [===>..........................] - ETA: 1s - loss: 0.3615 - accuracy: 0.8508
Epoch 00082: saving model to checkpoints\weights.82.hdf5
211/854 [======>.......................] - ETA: 1s - loss: 0.3605 - accuracy: 0.8507
Epoch 00082: saving model to checkpoints\weights.82.hdf5
302/854 [=========>....................] - ETA: 1s - loss: 0.3559 - accuracy: 0.8526
Epoch 00082: saving model to checkpoints\weights.82.hdf5
423/854 [=============>................] - ETA

854/854 [==============================] - 2s 2ms/step - loss: 0.3562 - accuracy: 0.8533
Epoch 88/100
  1/854 [..............................] - ETA: 1s - loss: 0.4131 - accuracy: 0.8438
Epoch 00088: saving model to checkpoints\weights.88.hdf5
 93/854 [==>...........................] - ETA: 1s - loss: 0.3479 - accuracy: 0.8562
Epoch 00088: saving model to checkpoints\weights.88.hdf5
186/854 [=====>........................] - ETA: 1s - loss: 0.3552 - accuracy: 0.8518
Epoch 00088: saving model to checkpoints\weights.88.hdf5
302/854 [=========>....................] - ETA: 1s - loss: 0.3591 - accuracy: 0.8517
Epoch 00088: saving model to checkpoints\weights.88.hdf5
393/854 [============>.................] - ETA: 1s - loss: 0.3628 - accuracy: 0.8489
Epoch 00088: saving model to checkpoints\weights.88.hdf5
482/854 [===============>..............] - ETA: 0s - loss: 0.3623 - accuracy: 0.8493
Epoch 00088: saving model to checkpoints\weights.88.hdf5
594/854 [===================>..........] - ETA

160/854 [====>.........................] - ETA: 1s - loss: 0.3570 - accuracy: 0.8547
Epoch 00094: saving model to checkpoints\weights.94.hdf5
273/854 [========>.....................] - ETA: 1s - loss: 0.3531 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.94.hdf5
363/854 [===========>..................] - ETA: 1s - loss: 0.3531 - accuracy: 0.8557
Epoch 00094: saving model to checkpoints\weights.94.hdf5
474/854 [===============>..............] - ETA: 0s - loss: 0.3527 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.94.hdf5
563/854 [==================>...........] - ETA: 0s - loss: 0.3536 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.94.hdf5
677/854 [======================>.......] - ETA: 0s - loss: 0.3556 - accuracy: 0.8541
Epoch 00094: saving model to checkpoints\weights.94.hdf5
767/854 [=========================>....] - ETA: 0s - loss: 0.3569 - accuracy: 0.8533
Epoch 00094: saving model to checkpoints\weights.94.hdf5
854/85

351/854 [===========>..................] - ETA: 1s - loss: 0.3527 - accuracy: 0.8540
Epoch 00100: saving model to checkpoints\weights.100.hdf5
443/854 [==============>...............] - ETA: 0s - loss: 0.3545 - accuracy: 0.8536
Epoch 00100: saving model to checkpoints\weights.100.hdf5
534/854 [=================>............] - ETA: 0s - loss: 0.3520 - accuracy: 0.8553
Epoch 00100: saving model to checkpoints\weights.100.hdf5
651/854 [=====================>........] - ETA: 0s - loss: 0.3543 - accuracy: 0.8535
Epoch 00100: saving model to checkpoints\weights.100.hdf5
740/854 [========================>.....] - ETA: 0s - loss: 0.3551 - accuracy: 0.8528
Epoch 00100: saving model to checkpoints\weights.100.hdf5
851/854 [============================>.] - ETA: 0s - loss: 0.3555 - accuracy: 0.8537
Epoch 00100: saving model to checkpoints\weights.100.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3554 - accuracy: 0.8538
285/285 - 0s - loss: 0.3993 - accuracy: 0.8449
Neural

In [12]:
# Define new model to test ModelCheckpoint reloading.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn_new = tf.keras.models.Sequential()

# Add layers.
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))
nn_new.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model.
nn_new.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load model from weights instead of training.
nn_new.load_weights('checkpoints/weights.100.hdf5')

# Evaluate the model using the test data.
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 0.3993 - accuracy: 0.8449
Loss: 0.39930155873298645, Accuracy: 0.8449373841285706
